In [67]:
from pathlib import Path
from typing import Dict
import shutil
import fileinput

import tabula
import pandas as pd
from tabula.template import load_template

In [15]:
PDF_FILE = "../data/pdfs/studis_fach/studis_fach-2010-winter.pdf"
ANNOTATION_JSON = Path("../data/annotations/tabula-2010-winter-studis-fach-abschluss.json")

assert Path(PDF_FILE).is_file()
assert Path(ANNOTATION_JSON).is_file()

In [79]:
ANNOTATION_DIR = Path("../annotations/")
PDFS_DIR = Path("../data/pdfs/studis_fach/")

In [89]:
PDFS_DIR

PosixPath('../data/pdfs/studis_fach')

In [90]:
x = sorted(list(ANNOTATION_DIR.glob("*.json")))

In [91]:
Path('../annotations/studis_fach-2007-sommer.json') in x

True

In [87]:
sorted(list(PDFS_DIR.glob("*.pdf")))

[PosixPath('../data/pdfs/studis_fach/studis_fach-2007-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2007-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2008-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2008-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2009-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2009-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2010-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2010-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2011-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2011-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2012-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2012-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2013-sommer.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach-2013-winter.pdf'),
 PosixPath('../data/pdfs/studis_fach/studis_fach

In [39]:
options = load_template(str(ANNOTATION_JSON))

In [43]:
tabula.convert_into(PDF_FILE, "out.csv", guess=False, output_format="csv", **options[0])

In [73]:
def convert_page_df(pdf_file, csv_file, options: Dict) -> None:
    tabula.convert_into(PDF_FILE, csv_file, guess=False, output_format="csv", **options)

In [76]:
def convert_pdf_with_annotation(pdf_file: Path, annotation_file: Path, csv_file: Path):
    assert csv_file.suffix == ".csv"
    options = load_template(str(annotation_file))
    
    tmp_dir = Path(".tmp")
    tmp_dir.mkdir(exist_ok=True)
    tmp_csv_files: List[Path] = [tmp_dir / f"{csv_file.stem}-{i}.csv" for i in range(len(options))]
    
    for page_options, tmp_csv_file in zip(options, tmp_csv_files):
        convert_page_df(str(pdf_file), str(tmp_csv_file), page_options)
        
    with csv_file.open(mode="w") as f:
        for line in fileinput.input(tmp_csv_files):
            f.write(line)
    
    shutil.rmtree(".tmp")
    

In [78]:
convert_pdf_with_annotation(Path(PDF_FILE), ANNOTATION_JSON, Path("test.csv"))